In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

from sklearn import linear_model
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd


In [2]:
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff

plotly.offline.init_notebook_mode(connected=True)

global weights1
global weights2

c:\users\home\appdata\local\programs\python\python35-32\lib\site-packages\plotly\tools.py:103: UserWarning:

Looks like you don't have 'read-write' permission to your 'home' ('~') directory or to our '~/.plotly' directory. That means plotly's python api can't setup local configuration files. No problem though! You'll just have to sign-in using 'plotly.plotly.sign_in()'. For help with that: 'help(plotly.plotly.sign_in)'.
Questions? Visit https://support.plot.ly



In [ ]:
def find_surrogate_model(Train_features,BlackBoxModel,mode):
    R2_Scores = [] #list to store R2 scores of different surrogate models and find maximum
    
    BlackBoxPred = BlackBoxModel.predict(Train_features) #Prediction of BlackBoxModel
    
    #first surrogate model is Decsiosn Tree
    if(mode == 'classification'):
        Model_DTree = tree.DecisionTreeClassifier() #decision tree classifier,if mode is classification
        Model_DTree.fit(Train_features,BlackBoxPred)
    
        R2_Scores.append(r2_score(BlackBoxPred,Model_DTree.predict(Train_features)))
    elif(mode == 'regression'):
        Model_DTree = tree.DecisionTreeRegressor() #decision tree regressor, if mode is regression
        Model_DTree.fit(Train_features,BlackBoxPred)
        
        R2_Scores.append(r2_score(BlackBoxPred,Model_DTree.predict(Train_features)))
            
    #second surrogate model is linear regression
    Model_LinReg = linear_model.LinearRegression()
    Model_LinReg.fit(Train_features,BlackBoxPred)
    
    R2_Scores.append(r2_score(BlackBoxPred,Model_DTree.predict(Train_features)))
    
    #third surrogat model is logistic regression
    Model_LogReg = linear_model.LogisticRegression()
    Model_LogReg.fit(Train_features,BlackBoxPred)
    
    R2_Scores.append(r2_score(BlackBoxPred,Model_DTree.predict(Train_features)))
    
    Max_R2Score = max(R2_Scores)
    SurrogateModels = [Model_DTree,Model_LinReg,Model_LogReg] #list to store and return all the models
    return R2_Scores.index(Max_R2Score)+1,SurrogateModels # index of final surrogate model and the list of models returned
    

the interpretation code

In [6]:
def interpret(Train_features,BlackBoxModel,feature_names,class_names,mode):
    
    SurrogateModel_index,Models = find_surrogate_model(Train_features,BlackBoxModel,mode)
    
    global weights1 
    weights1 = Models[1].coef_   # weights from Linear regression to display feature plot
    
    if(SurrogateModel_index == 0):
            SurrogateModel = Models[0]
            graph = Source(tree.export_graphviz(SurrogateModel, out_file=None
           , feature_names=feature_names, class_names=class_names
           , filled = True))
            display(SVG(graph.pipe(format='svg')))
            print("Starting from the root node you go to the next nodes and the edges tell you which subsets you are looking at. Once you reach the leaf node, the node tells you the predicted outcome. All the edges are connected by ‘AND’.")
    
    
    
    elif(SurrogateModel_index == 1):
            
            SurrogateModel = Models[1]
            
            d = {'Feature Names': feature_names,'Weights': weights1}
            d = pd.DataFrame(data = d)
            table = ff.create_table(d)
            plotly.offline.iplot(table, filename='pandas_table')
            plotly.offline.iplot({
            "data": [go.Box(x= feature_names, y= weights1)],
            "layout": go.Layout(title="Weight Plot for Linear Regression")
            })
    else:
            SurrogateModel = Models[2]
        
            global weights2
            weights2 = SurrogateModel.coef_[0]
        
            d = {'Feature Names': feature_names,'Weights': weights2,'Odds Ratio': np.exp(weights2)}
            d = pd.DataFrame(data = d)
            table = ff.create_table(d)
            plotly.offline.iplot(table, filename='pandas_table')
        
            plotly.offline.iplot({
            "data": [go.Box(x= feature_names, y= np.exp(weights2))],
            "layout": go.Layout(title="Weight Plot for Logistic Regression")
            })
        

function to show effect plot for each feature.

In [1]:
def show_feature_plot(Train_features,feature_names):
    effect_ij = []
    data = []
    #odds_ratio = np.exp(weights2)
    for i in range(len(feature_names)):
        for j in range (len(Train_features)):
            effect_ij.append(Train_features[j][i]*(weights1[i]))
        data.append(go.Box(x=effect_ij,name = feature_names[i]))
        effect_ij = []
    plotly.offline.iplot(data)
    '''for i in range(len(feature_names)):
        Sum = 0
        for j in range (len(Train_features)):
            Sum=Sum + Train_features[j][i]*(weights1[i])
        effect_ij.append(Sum)
    plotly.offline.iplot({
        "data": [go.Bar(x= feature_names,y=effect_ij)],
        "layout": go.Layout(title= "Effect Plot")
    })'''
                 